# Building your first model with `fastai`

In [1]:
# Import libraries
import torch
from fastai.vision.all import *

In [2]:
# Verify the default device is an NVIDIA GPU
print(torch.cuda.get_device_name(0))

GeForce RTX 2070 with Max-Q Design


### Downloading the PETS built-in dataset from fastai

To download data from fastai, we need to use the `untar_data()` function.
```python
untar_data(url, fname=None, dest=None, c_key='data', force_download=False, extract_func=file_extract, timeout=4)
```

The keyword `url` accepts a WINDOWS Path returned by `URLs.[dataset_name]`.

```python
# Pass the URLs.PETS path to download the pets dataset.
untar_data(URLs.PETS)
# Path on local machine to download/load the data
>>> Path('C:/Users/chris/.fastai/data/oxford-iiit-pet')
```

Within the WINDOWS path returned by `untar_data`, subpaths exist:
- Path('C:/Users/chris/.fastai/data/oxford-iiit-pet/annotations')
- Path('C:/Users/chris/.fastai/data/oxford-iiit-pet/images') <--- _The path that contains dog and cat images_


A complete list of built-in datasets: https://docs.fast.ai/data.external.html

In [4]:
# Load images from the PETS dataset
path = untar_data(URLs.PETS)/'images'

# Create a dataset of Dog and Cat images
dbunch = ImageDataLoaders.from_name_func(path,
                                         # Get image files in path recursively, only in folders, if specified.
                                         get_image_files(path),
                                         # Random 20% of the dataset is reserved for validation
                                         valid_pct=0.2,
                                         # Function is applied to the name of each file
                                         label_func=lambda x: x[0].isupper(),
                                         # Resize the images to 224
                                         item_tfms=Resize(224),
                                         # Random seed for reproducibility
                                         seed=369,
                                         # Device to use
                                         device='cuda',
                                         # Set to 0 in WINDOWS environment
                                         num_workers=0
                                        )
# Pass the dataset to a model pretrained on 1.3 million images.
learn = cnn_learner(dbunch, resnet34, metrics=error_rate)

# The resnet model will be tuned to recognize dogs and cats.
learn.fine_tune(2)

epoch,train_loss,valid_loss,error_rate,time
0,0.148423,0.038504,0.010149,00:59


epoch,train_loss,valid_loss,error_rate,time
0,0.038097,0.054823,0.010825,01:04
1,0.032240,0.017393,0.006766,01:03
